In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
students = pd.DataFrame([1,1])

def selectData(students: pd.DataFrame) -> pd.DataFrame:
    return students.shape

selectData(students)

(2, 1)

In [3]:
print(selectData(students))

(2, 1)


In [ ]:
def createBonusColumn(students: pd.DataFrame) -> pd.DataFrame:
    students['bonus'] = students['salary'] * 2
    return students

In [12]:
conn = sqlite3.connect(':memory:')
conn1 = sqlite3.connect('db.db')

In [13]:
cursor = conn.cursor()

In [23]:
cursor.execute("""
    INSERT INTO students (name, class)
    VALUES (?, ?)
""", ("Alice", "Math"))

In [29]:
cursor.execute("""PRAGMA table_info(students)""")

In [30]:
print(cursor.fetchall())

[(0, 'id', '', 0, None, 1), (1, 'name', 'VARCHAR', 0, None, 0), (2, 'class', 'VARCHAR', 0, None, 0)]


In [7]:
students = pd.DataFrame([(1,1),(2,1)])

In [32]:
selectData(students)

(2, 2)

In [35]:
students = pd.DataFrame([[1,1],[2,2],[3,3]])

In [36]:
students.shape

(3, 2)